In [42]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,f1_score


Classification

In [39]:
#加载数据
os.getcwd()
data = pd.read_csv(os.getcwd()+'\\time_resampling\\featured_time_resamping_with_moodlevel.csv')
data['time'] = pd.to_datetime(data['time'])

###############
#所有mood值向前移动一行
data['mood'] = data.groupby('id')['mood'].shift(-1)

# 按'id'和'time'排序，确保每个'id'分组的最后一行是时间最晚的
data.sort_values(by=['id', 'time'], inplace=True)

# 删除每个'id'分组的最后一行
data = data.groupby('id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)



features = data.drop(columns=['mood','id','time',"mood_level"])
target = data['mood_level']

Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size=0.3)


C:\Users\sydwi\AppData\Local\Temp\ipykernel_25984\3953763036.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


In [40]:

rt_classifier = RandomForestClassifier(random_state=1)
param_grid = {
    'n_estimators': [50, 100, 200],  # 树的数量
    'max_features': ['auto', 'sqrt', 'log2'],  # 考虑的最大特征数
    'max_depth': [None, 10, 20, 30],  # 最大树深度
    'min_samples_split': [2, 5, 10],  # 分割内部节点所需的最小样本数
    'min_samples_leaf': [1, 2, 4]  # 叶节点所需的最小样本数
}
grid_search = GridSearchCV(estimator=rt_classifier, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=2)

# 拟合网格搜索
grid_search.fit(Xtrain, Ytrain)
print("Best parameters found: ", grid_search.best_params_)
best_rf = grid_search.best_estimator_
val_predictions = best_rf.predict(Xtest)


Fitting 3 folds for each of 324 candidates, totalling 972 fits


c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
152 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  Fi

Best parameters found:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [45]:
f1 = f1_score(Ytest, val_predictions, average='weighted')  #计算f1值
print(f'F1 Score: {f1}')

F1 Score: 0.820462398825193


Regression

In [35]:

#加载数据
os.getcwd()
data = pd.read_csv(os.getcwd()+'\\time_resampling\\featured_time_resamping_sparse_matrix_data.csv')
data['time'] = pd.to_datetime(data['time'])

###############
#所有mood值向前移动一行
data['mood'] = data.groupby('id')['mood'].shift(-1)

# 按'id'和'time'排序，确保每个'id'分组的最后一行是时间最晚的
data.sort_values(by=['id', 'time'], inplace=True)

# 删除每个'id'分组的最后一行
data = data.groupby('id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
################
#data=data.drop(columns=['Unnamed: 0'])


# 提取年、月、日、星期几、小时等作为新特征
# data['year'] = data['time'].dt.year
# data['month'] = data['time'].dt.month
# data['day'] = data['time'].dt.day
# data['weekday'] = data['time'].dt.weekday
# data['hour'] = data['time'].dt.hour

# 使用正弦和余弦变换对月份和小时进行周期性编码
# data['sin_month'] = np.sin(2 * np.pi * data['month'] / 12)
# data['cos_month'] = np.cos(2 * np.pi * data['month'] / 12)
# data['sin_hour'] = np.sin(2 * np.pi * data['hour'] / 24)
# data['cos_hour'] = np.cos(2 * np.pi * data['hour'] / 24)


features = data.drop(columns=['mood','id','time'])
target = data['mood']

Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size=0.3)




C:\Users\sydwi\AppData\Local\Temp\ipykernel_25984\2302715881.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


In [36]:



# 实例化随机森林回归器
rf_regressor = RandomForestRegressor(random_state=1)
param_grid = {
    'n_estimators': [50, 100, 200],  # 树的数量
    'max_features': ['auto', 'sqrt', 'log2'],  # 考虑的最大特征数
    'max_depth': [None, 10, 20, 30],  # 最大树深度
    'min_samples_split': [2, 5, 10],  # 分割内部节点所需的最小样本数
    'min_samples_leaf': [1, 2, 4]  # 叶节点所需的最小样本数
}
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=2)

# 拟合网格搜索
grid_search.fit(Xtrain, Ytrain)
print("Best parameters found: ", grid_search.best_params_)
best_rf = grid_search.best_estimator_
val_predictions = best_rf.predict(Xtest)





Fitting 3 folds for each of 324 candidates, totalling 972 fits


c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
73 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  Fil

Best parameters found:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}


In [37]:


mse = mean_squared_error(Ytest, val_predictions)
print(f'Mean Squared Error (MSE): {mse}')

# 计算RMSE
rmse = mean_squared_error(Ytest, val_predictions, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# 计算MAE
mae = mean_absolute_error(Ytest, val_predictions)
print(f'Mean Absolute Error (MAE): {mae}')

# 计算R^2
r2 = r2_score(Ytest, val_predictions)
print(f'R^2: {r2}')

Mean Squared Error (MSE): 0.03134041949116926
Root Mean Squared Error (RMSE): 0.17703225551059687
Mean Absolute Error (MAE): 0.11304387503801658
R^2: 0.16745247019541631


c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
